In [176]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV


In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/04/28 12:25:27 WARN Utils: Your hostname, emil-GL552VW resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp2s0)
24/04/28 12:25:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/28 12:25:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [9]:
path = "../../../../data/yelp_dataset/yelp_academic_dataset_"

In [13]:
users = spark.read.format("json").load(f"{path}user.json")

In [ ]:
reviews = spark.read.format("json").load(f"{path}review.json")

In [21]:
business = spark.read.format("json").load(f"{path}business.json")

24/04/28 13:01:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [14]:
users.count()

1987897

In [15]:
users.show()

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

In [19]:
reviews.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [ ]:
reviews.count()

In [22]:
business.show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

In [ ]:
business.count()

# Data Engineering

In [114]:
bussines_cols = [
    "business_id",
    "city",
    "name",
]

reviews_cols = [
    "business_id",
    "user_id",
    "stars",
    "date",
    "review_id"
]

In [117]:
pub_business = business.filter(
    (F.col("categories").like("%Bar%")) | (F.col("categories").like("%Pubs%"))
).select(
    *bussines_cols,
    F.col("stars").alias("business_stars")
)

reviews_since_2016 = reviews.select(*reviews_cols, F.year(F.col("date")).alias("year")).filter("year > 2015")

columns_for_train_latent_variables = [
    "user_id",
    "business_id",
    "stars"
]

In [111]:
reviews

DataFrame[business_id: string, user_id: string, stars: double, date: string, year: int]

In [51]:
pub_business.show(truncate=False)

+----------------------+-------------+------------------------------------------+-----+
|business_id           |city         |name                                      |stars|
+----------------------+-------------+------------------------------------------+-----+
|k0hlBqXX-Bt0vf1op7Jr1w|Affton       |Tsevi's Pub And Grill                     |3.0  |
|MUTTqe8uqyMdBl186RmNeA|Philadelphia |Tuna Bar                                  |4.0  |
|WKMJwqnfZKsAae75RMP6jA|Edmonton     |Roast Coffeehouse and Wine Bar            |4.0  |
|QdN72BWoyFypdGJhhI5r7g|Philadelphia |Bar One                                   |4.0  |
|lk9IwjZXqUMqqOhM774DtQ|Nashville    |Caviar & Bananas                          |3.5  |
|txyXRytGjwOXvS8s4sc-WA|Tucson       |Smoothie King                             |3.0  |
|seKihQKpGGnCeLuELRQPSQ|Indianapolis |Twin Peaks                                |3.5  |
|L_TT0BFmFwORAMaA83K54A|Indianapolis |Village Tap Room                          |2.5  |
|qfWJmJ0g96eM_fWma3ja0g|West Che

In [120]:
pub_business_with_reviews = pub_business.join(reviews_since_2016, on="business_id", how="inner")

In [76]:
pub_business_with_reviews.select(F.max(F.col("year")), F.min(F.col("year"))).show()

+---------+---------+
|max(year)|min(year)|
+---------+---------+
|     2022|     2019|
+---------+---------+



In [78]:
pub_business_with_reviews.show()

+--------------------+----------------+--------------------+--------------+--------------------+-----+-------------------+----+
|         business_id|            city|                name|business_stars|             user_id|stars|               date|year|
+--------------------+----------------+--------------------+--------------+--------------------+-----+-------------------+----+
|M0r9lUn2gLFYgIwIf...|     Saint Louis|      Baileys' Range|           3.5|SRp90x9d2719GOZ_P...|  5.0|2019-01-06 11:48:21|2019|
|-3AooxIkg38UyUdlz...|   Santa Barbara|    Chase Restaurant|           3.0|NjuPidqqsNW9KG8PB...|  4.0|2019-02-17 20:28:26|2019|
|720J1v3TX-J95cdTJ...|       Nashville|               Ninki|           3.5|F8wJVQrOppgOwSXU0...|  5.0|2019-02-20 23:03:26|2019|
|dpwkz52tHB2MCPv53...|            Reno|             Jazmine|           3.5|z5YWFIhK7g59brxtl...|  1.0|2019-01-23 03:16:14|2019|
|UmjITdXHhEF46ho6I...|     New Orleans|     Dave & Buster's|           2.0|ARFM4LrI_zxogm1L2...|  3.0|20

In [132]:
review_by_business = pub_business_with_reviews.groupBy("business_id").count()

In [133]:
review_by_business

DataFrame[business_id: string, count: bigint]

In [136]:
review_by_business.select(F.percentile_approx("count", 0.25, )).show()

+-------------------------------------+
|percentile_approx(count, 0.25, 10000)|
+-------------------------------------+
|                                   11|
+-------------------------------------+



In [137]:
business_to_take = review_by_business.filter("count > 11")

In [139]:
pub_business_with_reviews = pub_business_with_reviews.join(business_to_take.select("business_id"), on="business_id", how="inner")

In [140]:
pub_business_with_reviews.count()

1248597

In [141]:
train = pub_business_with_reviews.sample(fraction=0.6, seed=1)

test_and_validation = pub_business_with_reviews.join(train, on="review_id", how="left_anti")

test = test_and_validation.sample(fraction=0.4, seed=1)

validation = test_and_validation.join(test, on="review_id", how="left_anti")

# Write data

In [151]:
train.coalesce(1).write.option("header", True).csv("../../data/train")
test.coalesce(1).write.option("header", True).csv("../../data/test")
validation.coalesce(1).write.option("header", True).csv("../../data/validation")

In [144]:
validation.show()

+--------------------+--------------------+--------------+--------------------+--------------+--------------------+-----+-------------------+----+
|           review_id|         business_id|          city|                name|business_stars|             user_id|stars|               date|year|
+--------------------+--------------------+--------------+--------------------+--------------+--------------------+-----+-------------------+----+
|--GMf2NgXFy5MF2Ls...|X3jaJDlA5YqN1EZhV...|St. Petersburg|    Red Mesa Mercado|           4.0|ic65syJ4uhzhmTpR9...|  4.0|2019-01-07 02:01:43|2019|
|--HR828qO-SSsjgU6...|49C1QPz9uxD-UyuYw...|   New Orleans|Deanie's Restaura...|           4.0|j_DWaYRCXBrrVbOVl...|  5.0|2019-05-01 20:13:03|2019|
|--P0WQ3B9MPoVKlUQ...|Mfvk9uEEhdCqj8S2u...|   Cherry Hill|          Seasons 52|           4.0|HqHZQEnaFH5D3cyuv...|  5.0|2016-12-23 21:38:48|2016|
|--RsqK4awL5fYKTzZ...|916Ngv5oF4mQEs7ou...|      Franklin|        BB's Bar-B-Q|           3.0|ZCc85CfjFaw5v24yo...|  3

# Read Data

In [183]:
train_pd = pd.read_csv("../../data/train.csv")
test_pd = pd.read_csv("../../data/test.csv")
validation_pd = pd.read_csv("../../data/validation.csv")

# Generate Surprise DataFrame

In [155]:
reader = Reader(rating_scale=( 1, 5))

In [159]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749128 entries, 0 to 749127
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   business_id     749128 non-null  object 
 1   city            749128 non-null  object 
 2   name            749128 non-null  object 
 3   business_stars  749128 non-null  float64
 4   user_id         749128 non-null  object 
 5   stars           749128 non-null  float64
 6   date            749128 non-null  object 
 7   review_id       749128 non-null  object 
 8   year            749128 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 51.4+ MB


In [184]:
train_dataset = Dataset.load_from_df(train_pd[["user_id", "business_id", "stars"]], reader)
test = Dataset.load_from_df(test_pd[["user_id", "business_id", "stars"]], reader)
validation = Dataset.load_from_df(validation_pd[["user_id", "business_id", "stars"]], reader)

In [161]:
train

In [162]:
train = train_dataset.build_full_trainset()
validation = validation.build_full_trainset()
test = test.build_full_trainset()

In [163]:
train_test = train.build_testset()
validation = validation.build_testset()
test = test.build_testset()

In [166]:
algo = SVD(
    n_factors=5,
    n_epochs=200,
    biased=True, 
    lr_all=0.001,
    reg_all=0,
    init_mean=0,
    init_std_dev=0.01,
    verbose=True
)

In [178]:
param_grid = {
    "n_epochs": [50, 100, 150, 200],
    "lr_all": [0.001,0.002, 0.005],
    "n_factors": [3, 5, 7, 10],
    "biased": [True, False]
}

grid_search = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)

In [186]:
grid_search.fit(train_dataset)

In [ ]:

print(grid_search.best_score["rmse"])

In [167]:
algo.fit(train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [169]:
predictions_train = algo.test(train_test)
predictions_test = algo.test(test)
predictions_validation = algo.test(validation)

In [170]:
accuracy.rmse(predictions_train, verbose=True)

RMSE: 1.0220


1.021950853275051

In [171]:
accuracy.rmse(predictions_validation, verbose=True)

RMSE: 1.2192


1.21915309676405

In [172]:
accuracy.rmse(predictions_test, verbose=True)

RMSE: 1.2191


1.2190771163887784

In [173]:
accuracy.mae(predictions_train, verbose=True)

MAE:  0.8100


0.8100028687458145

In [174]:
accuracy.mae(predictions_test, verbose=True)

MAE:  0.9720


0.9719598333533005

In [175]:
accuracy.mae(predictions_validation, verbose=True)

MAE:  0.9713


0.9713019697871417